In [24]:
import json
import pickle
import random

import numpy as np
import nltk
from nltk.stem import WordNetLemmatizer

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import SGD

In [25]:
nltk.download('punkt')
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt to C:\Users\Abdullah
[nltk_data]     Tariq/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to C:\Users\Abdullah
[nltk_data]     Tariq/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [26]:
lemmatizer = WordNetLemmatizer()
intents = json.load(open("intents.json", "rb"))

In [27]:
words = []
classes = []
documents = []
ignore_letters = ['?', '!', '.', ',']

for intent in intents["intents"]:
    for patterns in intent["patterns"]:
        word_list = nltk.word_tokenize(patterns)
        words.extend(word_list)
        documents.append((word_list, intent["tag"]))
        if intent["tag"] not in classes:
            classes.append(intent["tag"])
            

In [28]:
words = [lemmatizer.lemmatize(word.lower()) for word in words if word not in ignore_letters]
words = sorted(set(words))
classes = sorted(set(classes))

In [29]:
pickle.dump(words, open("words.pkl", "wb"))
pickle.dump(classes, open("classes.pkl", "wb"))

In [30]:
training = []
output_empty = [0]*len(classes)

for document in documents:
    bag = []
    word_patterns = document[0]
    word_patterns = [lemmatizer.lemmatize(word.lower()) for word in word_patterns]
    for word in words:
        bag.append(1) if word in word_patterns else bag.append(0)
        
    output_row = list(output_empty)
    output_row[classes.index(document[1])] = 1
    training.append([bag, output_row])

In [31]:
random.shuffle(training)

X = np.array([item[0] for item in training], dtype=np.float32)
Y = np.array([item[1] for item in training], dtype=np.float32)

In [32]:
Y.shape

(266, 30)

In [34]:
model = Sequential([
    Dense(128, input_shape=(X.shape[1], ), activation="relu"),
    Dropout(0.5),
    Dense(64, activation="relu"),
    Dropout(0.5),
    Dense(Y.shape[1], activation="softmax")
])

c:\Users\Abdullah Tariq\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [35]:
sgd = SGD(learning_rate=0.01, weight_decay=1e-6, momentum=0.9, nesterov=True)
model.compile(optimizer=sgd, loss="categorical_crossentropy", metrics=["accuracy"])

In [36]:
model.fit(X, Y, epochs=200, batch_size=10, verbose=1)

Epoch 1/200
27/27 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.0208 - loss: 3.4206  
Epoch 2/200
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.0402 - loss: 3.3759   
Epoch 3/200
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.0782 - loss: 3.3632   
Epoch 4/200
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.1261 - loss: 3.2700
Epoch 5/200
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.1002 - loss: 3.2793
Epoch 6/200
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.1498 - loss: 3.2286
Epoch 7/200
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.1613 - loss: 3.1862
Epoch 8/200
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.1682 - loss: 3.1359
Epoch 9/200
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.1846 - loss: 2.9618
Epoch 10/200
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.2263 - loss: 2.9368
Epoch 11/200
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.3178 - loss: 2.7622
Epoch 12/200
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - a

In [37]:
model.save("Chatbot.h5")